In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("/kaggle/input/hmcdataset/intraday.csv")



In [ ]:
# Preprocessing
df['start'] = pd.to_datetime(df['start'], errors='coerce')
df['date'] = df['start'].dt.date
df['hour'] = df['start'].dt.floor('h')  # use lowercase 'h' to avoid deprecation warning

In [ ]:


# Filter relevant columns and define hypoglycemia
df_cgm = df[['patientID', 'date', 'hour', 'cgm', 'steps']].dropna(subset=['cgm'])
df_cgm['hypo'] = df_cgm['cgm'] < 70

# 1. Total number of patients
total_patients = df_cgm['patientID'].nunique()

# 2. Average number of days per patient
days_per_patient = df_cgm.groupby('patientID')['date'].nunique()
avg_days_per_patient = days_per_patient.mean()

# 3. Average valid CGM hours per patient (≥ 4 readings/hour)
valid_hours = df_cgm.groupby(['patientID', 'hour'])['cgm'].count().reset_index()
valid_hours = valid_hours[valid_hours['cgm'] >= 4]
valid_cgm_hours_per_patient = valid_hours.groupby('patientID')['hour'].count()
avg_valid_hours_per_patient = valid_cgm_hours_per_patient.mean()

# 4. Hypoglycemia hours per patient
hypo_hours_per_patient = df_cgm[df_cgm['hypo']].groupby('patientID')['hour'].nunique()
min_hypo = hypo_hours_per_patient.min()
max_hypo = hypo_hours_per_patient.max()
avg_hypo = hypo_hours_per_patient.mean()

# 5. Average steps per patient (interval-based)
avg_steps_per_patient = df_cgm.groupby('patientID')['steps'].mean().mean()

# 6. Average steps before hypo event (6h window)
df_cgm_sorted = df_cgm.sort_values(['patientID', 'hour']).set_index('hour')
df_cgm_sorted['prev_6h_steps'] = df_cgm_sorted.groupby('patientID')['steps'].rolling('6h').sum().reset_index(level=0, drop=True)
df_cgm_sorted.reset_index(inplace=True)

hypo_step_window = df_cgm_sorted[df_cgm_sorted['hypo']]['prev_6h_steps']
avg_steps_before_hypo = hypo_step_window.mean()

# Round results to nearest thousandth
summary_data = {
    "Metric": [
        "Total Patients",
        "Average Days per Patient",
        "Average Valid CGM Hours per Patient (≥4/hr)",
        "Min Hypo Hours per Patient",
        "Max Hypo Hours per Patient",
        "Average Hypo Hours per Patient",
        "Average Steps per Patient (interval level)",
        "Average Steps Before Hypo Event (6h window)"
    ],
    "Value": [
        round(total_patients, 3),
        round(avg_days_per_patient, 3),
        round(avg_valid_hours_per_patient, 3),
        int(min_hypo),
        int(max_hypo),
        round(avg_hypo, 3),
        round(avg_steps_per_patient, 3),
        round(avg_steps_before_hypo, 3)
    ]
}

summary_df = pd.DataFrame(summary_data)

# Print clean table
print("\nRounded CGM Summary Statistics:")
print(summary_df.to_string(index=False))


In [ ]:


# Aggregate to daily level: steps total, hypo % per patient per day
daily_summary = df_cgm.groupby(['patientID', 'date']).agg(
    total_steps=('steps', 'sum'),
    hypo_percent=('hypo', 'mean')
).reset_index()

# Define hypo_day if ≥ 4% CGM readings < 70
daily_summary['hypo_day'] = daily_summary['hypo_percent'] >= 0.04

# Per-patient summary (QCRI harmonized)
qcri_summary = daily_summary.groupby('patientID').agg(
    total_days=('date', 'nunique'),
    hypo_days=('hypo_day', 'sum'),
    avg_daily_steps=('total_steps', 'mean')
).reset_index()

qcri_summary['hypo_day_percent'] = round((qcri_summary['hypo_days'] / qcri_summary['total_days']) * 100, 3)
qcri_summary['avg_daily_steps'] = round(qcri_summary['avg_daily_steps'], 3)

# Prepare validation table
validation_table = pd.DataFrame({
    "Metric": [
        "Total Days (avg)",
        "% Hypoglycemic Days (range)",
        "Avg. Daily Steps (range)"
    ],
    "Your Computation (Per QCRI Style)": [
        round(qcri_summary['total_days'].mean(), 3),
        f"{qcri_summary['hypo_day_percent'].min()}% to {qcri_summary['hypo_day_percent'].max()}%",
        f"{qcri_summary['avg_daily_steps'].min()} to {qcri_summary['avg_daily_steps'].max()}"
    ]
})


# Prepare validation table
validation_table = pd.DataFrame({
    "Metric": [
        "Total Days (avg)",
        "% Hypoglycemic Days (range)",
        "Avg. Daily Steps (range)"
    ],
    "Your Computation (Per QCRI Style)": [
        round(qcri_summary['total_days'].mean(), 3),
        f"{qcri_summary['hypo_day_percent'].min()}% to {qcri_summary['hypo_day_percent'].max()}%",
        f"{qcri_summary['avg_daily_steps'].min()} to {qcri_summary['avg_daily_steps'].max()}"
    ]
})

# Display table in console
print("\\nQCRI-Style Validation Table:")
print(validation_table.to_string(index=False))


import warnings
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress FutureWarning messages from seaborn/pandas
warnings.simplefilter(action='ignore', category=FutureWarning)

# Boxplot: Daily Steps on Hypo vs Non-Hypo Days
plt.figure(figsize=(10, 6))
sns.boxplot(data=daily_summary, x='hypo_day', y='total_steps')
plt.xticks([0, 1], ['Non-Hypo Day', 'Hypo Day'])
plt.title('Boxplot: Daily Step Count on Hypo vs Non-Hypo Days')
plt.xlabel('')
plt.ylabel('Total Daily Steps')
plt.tight_layout()
plt.show()

# Histogram: % Hypoglycemic Days per Patient
plt.figure(figsize=(10, 6))
sns.histplot(qcri_summary['hypo_day_percent'], bins=15, kde=True)
plt.axvline(38, color='red', linestyle='--', label='QCRI Max Threshold (38%)')
plt.title('Histogram: % of Hypoglycemic Days per Patient')
plt.xlabel('Hypoglycemic Days (%)')
plt.ylabel('Patient Count')
plt.legend()
plt.tight_layout()
plt.show()


# Create a bar plot showing each patient's percentage of hypoglycemic days
plt.figure(figsize=(12, 6))
sns.barplot(data=qcri_summary, x='patientID', y='hypo_day_percent', palette='viridis')

plt.title('Percentage of Hypoglycemic Days per Patient')
plt.xlabel('Patient ID')
plt.ylabel('Hypoglycemic Days (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# Extract hour from datetime and calculate hypoglycemia rate by hour of day
df['start'] = pd.to_datetime(df['start'], errors='coerce')
df['hour_of_day'] = df['start'].dt.hour
df['hypo'] = df['cgm'].lt(70) & df['cgm'].notna()


# Group by hour of day and compute hypoglycemia frequency
hourly_hypo = df.dropna(subset=['cgm']).groupby('hour_of_day')['hypo'].mean().reset_index()

# Plot hypoglycemia frequency by hour of day
plt.figure(figsize=(12, 6))
sns.lineplot(data=hourly_hypo, x='hour_of_day', y='hypo', marker='o')
plt.title('Hourly Hypoglycemia Frequency Across All Patients')
plt.xlabel('Hour of Day (0 = Midnight)')
plt.ylabel('Hypoglycemia Rate')
plt.xticks(range(0, 24))
plt.grid(True)
plt.tight_layout()
plt.show()


# Group by hour of day and compute CGM hypo-related statistics
hourly_stats = df.groupby('hour_of_day').agg(
    hypo_min=('hypo', 'min'),
    hypo_max=('hypo', 'max'),
    hypo_mean=('hypo', 'mean'),
    hypo_std=('hypo', 'std')
).reset_index()

# Plot hypo stats over the day
plt.figure(figsize=(14, 6))
sns.lineplot(data=hourly_stats, x='hour_of_day', y='hypo_mean', label='Mean Hypo Rate', marker='D')
sns.lineplot(data=hourly_stats, x='hour_of_day', y='hypo_std', label='Std Dev Hypo Rate', marker='^')

plt.title('Hourly Hypoglycemia Frequency Across All Patients')
plt.xlabel('Hour of Day (0 = Midnight)')
plt.ylabel('Hypoglycemia Rate (0.01 to 0.3)')
plt.xticks(range(0, 24))
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()



# Group by hour of day and compute CGM statistics
hourly_stats = df.groupby('hour_of_day').agg(
    cgm_min=('cgm', 'min'),
    cgm_max=('cgm', 'max'),
    cgm_mean=('cgm', 'mean'),
    cgm_std=('cgm', 'std')
).reset_index()

# Plot all on same diagram with different colors
plt.figure(figsize=(14, 6))
sns.lineplot(data=hourly_stats, x='hour_of_day', y='cgm_mean', label='Mean CGM', marker='D')
sns.lineplot(data=hourly_stats, x='hour_of_day', y='cgm_std', label='Std Dev CGM', marker='^')
sns.lineplot(data=hourly_stats, x='hour_of_day', y='cgm_min', label='min CGM', marker='^')
plt.title('Hourly CGM Statistics Across All Patients')
plt.xlabel('Hour of Day (0 = Midnight)')
plt.ylabel('CGM Value (50 to 200)')
plt.xticks(range(0, 24))
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:


# Keep only relevant columns and drop rows without CGM
df_cgm = df[['patientID', 'hour', 'cgm']].dropna(subset=['cgm'])

# STEP 1: Filter for complete hours (≥ 4 CGM readings)
grouped = df_cgm.groupby(['patientID', 'hour'])
valid_hours = grouped.filter(lambda x: len(x) >= 4)

# STEP 2: Create features and label
features = valid_hours.groupby(['patientID', 'hour']).agg(
    cgm_std=('cgm', 'std'),
    cgm_min=('cgm', 'min'),
   cgm_mean=('cgm', 'mean'),
    cgm_max=('cgm', 'max'),
   
    hypo_label=('cgm', lambda x: int((x < 70).any()))
).reset_index()

# STEP 3: Sort for time-series modeling
features = features.sort_values(['patientID', 'hour']).reset_index(drop=True)

# STEP 4: Display preview of the processed data
print("LSTM-ready features (preview):")
print(features.head())


In [ ]:
# Define thresholds for each CGM stat
thresholds = {
    'cgm_min': [60, 70, 80, 90],
    'cgm_max': [120, 150, 180, 200],
    'cgm_mean': [70, 90, 110, 130],
    'cgm_std': [10, 20, 30, 40]
}

# Create a summary of hypo_label distribution by threshold
threshold_summary = []

for feature, values in thresholds.items():
    for thresh in values:
        below_thresh = features[features[feature] < thresh]
        hypo_0 = (below_thresh['hypo_label'] == 0).sum()
        hypo_1 = (below_thresh['hypo_label'] == 1).sum()
        threshold_summary.append({
            'Feature': feature,
            'Threshold': f"< {thresh}",
            'Label 0 Count': hypo_0,
            'Label 1 Count': hypo_1
        })

# Convert to DataFrame for display
threshold_summary_df = pd.DataFrame(threshold_summary)
threshold_summary_df


cgm_min < 70 is a perfect predictor of hypoglycemia in this dataset.

cgm_mean < 90 captures nearly 75% of hypoglycemic cases.

cgm_std values are higher in hypoglycemic hours, indicating variability.

In [ ]:
import numpy as np

# Use only cgm_mean without any scaling
feature_cols = ['cgm_mean']
sequence_length = 24

X_sequences = []
y_labels = []

# Group by patient and preserve time order
for patient_id, group in features.groupby('patientID'):
    group = group.sort_values('hour').reset_index(drop=True)
    for i in range(len(group) - sequence_length):
        sequence = group.loc[i:i+sequence_length-1, feature_cols].values
        label = group.loc[i + sequence_length, 'hypo_label']
        X_sequences.append(sequence)
        y_labels.append(label)

# Convert to numpy arrays for model use
X = np.array(X_sequences)  # shape: (samples, 6, 1)
y = np.array(y_labels)

# Confirm the shape
print("X shape:", X.shape)
print("y shape:", y.shape)


print(X[:5])
print(y[:20])


In [ ]:
!pip install tensorflow


In [ ]:
from tensorflow.keras.layers import Input

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf



# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Augmentation: Add Gaussian noise
def augment_time_series(X, y):
    noise = np.random.normal(0, 0.01, X.shape)
    X_aug = X + noise
    return np.vstack((X, X_aug)), np.hstack((y, y))

X_train_aug, y_train_aug = augment_time_series(X_train, y_train)

# Define one LSTM model (simplified from your template)
model = Sequential([
      Input(shape=(X.shape[1], X.shape[2])),  # replaces input_shape in LSTM
    LSTM(50, return_sequences=True),
    Dropout(0.3),
    LSTM(25),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_aug, y_train_aug, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Predict and evaluate
y_pred_prob = model.predict(X_test).flatten()
y_pred = (y_pred_prob >= 0.5).astype(int)

# Evaluation function
def evaluate_classification(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    auc_micro = roc_auc_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    print(f"📌 {model_name} - Classification Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"AUC Micro: {auc_micro:.4f}")

    # Confusion matrix plot
    plt.figure(figsize=(6, 4))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=[0, 1], yticklabels=[0, 1])
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Confusion Matrix - {model_name}")
    plt.show()

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "AUC Micro": auc_micro
    }

# Evaluate model
evaluation_results = evaluate_classification(y_test, y_pred, "LSTM_6hr")
evaluation_results


📊 Performance Summary
Metric	Value	Notes
Accuracy	95.96%	High overall accuracy, but misleading due to class imbalance.
Precision	0.75	75% of predicted hypoglycemic events were correct. Good! ✅
Recall	0.076	Only 7.6% of actual hypoglycemic events were detected. 🚨 Too low
F1-Score	0.138	Reflects poor balance between precision & recall.
AUC (Micro)	0.537	Just above random guessing (0.5) — underperforming globally.

🧠 Interpretation:
Your model is too conservative — it doesn't trigger hypoglycemia detection often enough.

It misses ~92% of actual hypoglycemic events, which is a critical failure in medical contexts.

However, when it does predict hypo, it’s fairly accurate (75% precision).



In [ ]:
# Count the number of samples for each class in the target variable `y`
from collections import Counter

label_distribution = Counter(y)
label_distribution


In [ ]:
from tensorflow.keras.layers import Input

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, roc_curve, precision_recall_curve, classification_report, auc
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1

# --- Assume X and y are already defined as numpy arrays or pandas DataFrame/Series ---
# Example placeholder:
# X = np.random.rand(1000, 10, 1)
# y = np.random.randint(0, 2, 1000)

# --- Evaluation Function ---
def evaluate(y_true, y_pred, threshold):
    print(f"\nThreshold: {threshold}")
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"Recall:    {recall_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"F1 Score:  {f1_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"AUC:       {roc_auc_score(y_true, y_pred):.4f}")

# --- Augment Function ---
def augment(X, y):
    noise = np.random.normal(0, 0.01, X.shape)
    return np.vstack((X, X + noise)), np.hstack((y, y))

# --- Train/Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train, y_train = augment(X_train, y_train)

# --- Model Definitions ---
def define_models(input_shape):
    return {
        "LSTM_100": Sequential([
            Input(shape=input_shape),
            LSTM(100, return_sequences=True),
            Dropout(0.2),
            LSTM(50),
            Dropout(0.2),
            Dense(25, activation='relu'),
            Dense(1, activation='sigmoid')
        ]),

        "LSTM_50": Sequential([
            Input(shape=input_shape),
            LSTM(50, return_sequences=True),
            Dropout(0.2),
            LSTM(25),
            Dropout(0.2),
            Dense(10, activation='relu'),
            Dense(1, activation='sigmoid')
        ]),

        "LSTM_25_L1": Sequential([
            Input(shape=input_shape),
            LSTM(50, return_sequences=True, kernel_regularizer=l1(0.01)),
            Dropout(0.2),
            LSTM(25, kernel_regularizer=l1(0.01)),
            Dropout(0.2),
            Dense(10, activation='relu', kernel_regularizer=l1(0.01)),
            Dense(1, activation='sigmoid')
        ])
    }

# --- Training and Evaluation ---
models = define_models((X.shape[1], X.shape[2]))
results = {}

for name, model in models.items():
    print(f"\nTraining {name} without FOCAL LOSS...")

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    y_pred_prob = model.predict(X_test).flatten()

    for threshold in [0.5, 0.4, 0.3, 0.2]:
        y_pred = (y_pred_prob >= threshold).astype(int)

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        auc_score = roc_auc_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)

        evaluate(y_test, y_pred, threshold)

        results[f"{name}_thr_{threshold}"] = {
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1,
            "AUC": auc_score
        }

        # Plot Confusion Matrix
        plt.figure(figsize=(5, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f"Confusion Matrix - {name} @ threshold {threshold}")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.tight_layout()
        plt.show()

# --- Summary Table ---
results_df = pd.DataFrame(results).T
print("\nFinal Evaluation Results:")
print(results_df.round(4))

# --- Optimal Threshold Analysis (using last model's y_pred_prob) ---
fpr, tpr, roc_thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)
youden_index = np.argmax(tpr - fpr)
best_roc_threshold = roc_thresholds[youden_index]

precision, recall, pr_thresholds = precision_recall_curve(y_test, y_pred_prob)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
best_f1_index = np.argmax(f1_scores)
best_f1_threshold = pr_thresholds[best_f1_index]

# --- Plot ROC & PR Curves ---
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
plt.scatter(fpr[youden_index], tpr[youden_index], color='red', label=f'Best threshold = {best_roc_threshold:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(recall, precision, label="PR curve")
plt.scatter(recall[best_f1_index], precision[best_f1_index], color='green', label=f'Best F1 threshold = {best_f1_threshold:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()

plt.tight_layout()
plt.show()

# --- Final Optimal Threshold Evaluation ---
best_threshold = best_f1_threshold
y_pred_opt = (y_pred_prob >= best_threshold).astype(int)

print(f"\nBest threshold by Youden's J (ROC): {best_roc_threshold:.4f}")
print(f"Best threshold by max F1-score (PR): {best_f1_threshold:.4f}")

print("\nClassification Report at Optimal Threshold:")
print(classification_report(y_test, y_pred_opt, digits=4))


📊 LSTM Model Comparison
Model	Accuracy	Precision	Recall	F1-Score	AUC
LSTM_100	0.9598	0.7209	0.0871	0.1554	0.5428
LSTM_50	0.9564	0.4746	0.2360	0.3152	0.6122
LSTM_25	0.9575	0.0000	0.0000	0.0000	0.5000

 Key Insights:
LSTM_50 has the best recall (23.6%) and F1-score (0.31) — it's the most balanced.

LSTM_100 has the highest precision (72%) but recall is very low — too conservative.

LSTM_25 failed to detect any positives at all. (100% false negatives)

Interpretation:
LSTM_50 is your best candidate so far for detecting hypoglycemia reliably.

LSTM_100 might be useful when false alarms are costly, but it misses most real events.

LSTM_25 likely over-regularized (due to L2 penalties), silencing detection entirely.

 
 Recommendations:
Stick with LSTM_50 — and tune it:

Try reducing dropout.

Train longer (e.g., 15 epochs).

Use threshold tuning (predict 1 if y_prob > 0.3 instead of 0.5).

Try a Hybrid Model: LSTM_50 + rule-based flag (cgm_min < 70).

Plot Precision-Recall curve or ROC AUC to visualize trade-offs.

In [ ]:
from tensorflow.keras.layers import Input

import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


def evaluate(y_true, y_pred, threshold):
    print(f"\nThreshold: {threshold}")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"F1 Score: {f1_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"AUC: {roc_auc_score(y_true, y_pred):.4f}")





# ✅ Focal Loss definition
def focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        return alpha_t * tf.math.pow((1 - p_t), gamma) * bce
    return loss


# ✅ Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# ✅ Augmentation: add small noise
def augment(X, y):
    noise = np.random.normal(0, 0.01, X.shape)
    return np.vstack((X, X + noise)), np.hstack((y, y))
X_train, y_train = augment(X_train, y_train)

# ✅ Define models
def define_models(input_shape):
    return {
        "LSTM_100": Sequential([
            Input(shape=input_shape),
            LSTM(100, return_sequences=True),
            Dropout(0.2),
            LSTM(50),
            Dropout(0.2),
            Dense(25, activation='relu'),
            Dense(1, activation='sigmoid')
        ]),

        "LSTM_50": Sequential([
            Input(shape=input_shape),
            LSTM(50, return_sequences=True),
            Dropout(0.2),
            LSTM(25),
            Dropout(0.2),
            Dense(10, activation='relu'),
            Dense(1, activation='sigmoid')
        ]),

        "LSTM_25_L1": Sequential([
            Input(shape=input_shape),
            LSTM(50, return_sequences=True, kernel_regularizer=l1(0.01)),
            Dropout(0.2),
            LSTM(25, kernel_regularizer=l1(0.01)),
            Dropout(0.2),
            Dense(10, activation='relu', kernel_regularizer=l1(0.01)),
            Dense(1, activation='sigmoid')
        ])
    }

# ✅ Train and evaluate using focal loss
models = define_models((X.shape[1], X.shape[2]))
results = {}

for name, model in models.items():
    print(f"Training {name}...")

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    y_pred_prob = model.predict(X_test).flatten()

    for threshold in [0.5, 0.4, 0.3, 0.2]:
        y_pred = (y_pred_prob >= threshold).astype(int)

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        auc = roc_auc_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)

        print(f"\n{name} @ threshold {threshold}")
        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

        # Save results (if needed only once per model or per threshold)
        results[f"{name}_thr_{threshold}"] = {
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1,
            "AUC": auc
        }

        # Confusion Matrix
        plt.figure(figsize=(5, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f"Confusion Matrix - {name} @ {threshold}")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.tight_layout()
        plt.show()

# Summary Table
results_df = pd.DataFrame(results).T
print("\nFinal Evaluation Results:")
print(results_df.round(4))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


In [ ]:
from tensorflow.keras.layers import Input

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.regularizers import l1
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, mean_squared_error, mean_absolute_error,
    roc_curve, precision_recall_curve, auc
)

# ✅ Focal Loss
def focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        return alpha_t * tf.pow(1 - p_t, gamma) * bce
    return loss

# ✅ Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# ✅ Augmentation
def augment(X, y):
    noise = np.random.normal(0, 0.01, X.shape)
    return np.vstack((X, X + noise)), np.hstack((y, y))

X_train, y_train = augment(X_train, y_train)

# ✅ Define Models
def define_models(input_shape):
    return {
        "LSTM_100": Sequential([
            Input(shape=input_shape),
            LSTM(100, return_sequences=True),
            Dropout(0.2),
            LSTM(50),
            Dropout(0.2),
            Dense(25, activation='relu'),
            Dense(1, activation='sigmoid')
        ]),

        "LSTM_50": Sequential([
            Input(shape=input_shape),
            LSTM(50, return_sequences=True),
            Dropout(0.2),
            LSTM(25),
            Dropout(0.2),
            Dense(10, activation='relu'),
            Dense(1, activation='sigmoid')
        ]),

        "LSTM_25_L1": Sequential([
            Input(shape=input_shape),
            LSTM(50, return_sequences=True, kernel_regularizer=l1(0.01)),
            Dropout(0.2),
            LSTM(25, kernel_regularizer=l1(0.01)),
            Dropout(0.2),
            Dense(10, activation='relu', kernel_regularizer=l1(0.01)),
            Dense(1, activation='sigmoid')
        ])
    }

# ✅ Evaluation Function
def evaluate_with_regression_metrics(y_true, y_pred, model_name="LSTM"):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auc_score = roc_auc_score(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    print(f"\n📊 {model_name} - Evaluation Metrics:")
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}")
    print(f"F1-Score: {f1:.4f}, AUC: {auc_score:.4f}")
    print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}")

    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.show()

    return {
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1-Score": f1,
        "AUC": auc_score,
        "RMSE": rmse,
        "MAE": mae
    }

# ✅ Training and Evaluation
models = define_models((X.shape[1], X.shape[2]))
results = {}

for name, model in models.items():
    print(f"\n🚀 Training {name} with FOCAL LOSS...")
    model.compile(optimizer='adam', loss=focal_loss(), metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    y_pred_prob = model.predict(X_test).flatten()

    # 📈 ROC and PR Curve Analysis (once per model)
    fpr, tpr, roc_thresholds = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    youden_index = np.argmax(tpr - fpr)
    best_roc_threshold = roc_thresholds[youden_index]

    precision, recall, pr_thresholds = precision_recall_curve(y_test, y_pred_prob)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_f1_index = np.argmax(f1_scores)
    best_f1_threshold = pr_thresholds[best_f1_index]

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
    plt.scatter(fpr[youden_index], tpr[youden_index], color='red', label=f'Best ROC Threshold = {best_roc_threshold:.2f}')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(recall, precision, label='PR Curve')
    plt.scatter(recall[best_f1_index], precision[best_f1_index], color='green', label=f'Best F1 Threshold = {best_f1_threshold:.2f}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()

    plt.tight_layout()
    plt.show()

    print(f"Best threshold by Youden's J statistic (ROC): {best_roc_threshold:.4f}")
    print(f"Best threshold by max F1-score (PR): {best_f1_threshold:.4f}")

    # 📊 Evaluate using fixed thresholds
    for threshold in [0.5, 0.4, 0.3, 0.2, best_f1_threshold]:
        y_pred = (y_pred_prob >= threshold).astype(int)
        metrics = evaluate_with_regression_metrics(y_test, y_pred, model_name=f"{name} @ {threshold:.2f}")
        results[f"{name}_thr_{threshold:.2f}"] = metrics

# ✅ Summary Table
results_df = pd.DataFrame(results).T
print("\n✅ Final Evaluation Results:")
print(results_df.round(4))


In [ ]:
from collections import Counter

# Count the class distribution
full_distribution = Counter(y)
train_distribution = Counter(y_train)
test_distribution = Counter(y_test)

# Display results clearly
print("🔢 Class Distribution Summary")
print(f"➡️ Full Dataset     : {dict(full_distribution)}")
print(f"➡️ Training Set     : {dict(train_distribution)}")
print(f"➡️ Testing Set      : {dict(test_distribution)}")

# Optionally, show class proportions
def class_proportions(counter):
    total = sum(counter.values())
    return {cls: f"{count} ({count/total:.2%})" for cls, count in counter.items()}

print("\n📊 Class Proportions")
print(f"Full    : {class_proportions(full_distribution)}")
print(f"Train   : {class_proportions(train_distribution)}")
print(f"Test    : {class_proportions(test_distribution)}")


In [ ]:
from tensorflow.keras.layers import Input

from sklearn.utils import resample
import numpy as np
from collections import Counter

# Flatten LSTM format for under-sampling
X_flat = X_train.reshape(X_train.shape[0], -1)

# Separate the classes
X_majority = X_flat[y_train == 0]
X_minority = X_flat[y_train == 1]

y_majority = y_train[y_train == 0]
y_minority = y_train[y_train == 1]

# Downsample majority class
X_majority_down, y_majority_down = resample(
    X_majority, y_majority,
    replace=False,
    n_samples=len(y_minority),
    random_state=42
)

# Combine balanced data
X_balanced = np.vstack((X_majority_down, X_minority))
y_balanced = np.hstack((y_majority_down, y_minority))

# Shuffle the balanced data
indices = np.random.permutation(len(y_balanced))
X_balanced = X_balanced[indices]
y_balanced = y_balanced[indices]

# Reshape back to LSTM format
X_balanced = X_balanced.reshape(-1, X_train.shape[1], X_train.shape[2])

# Report new distribution
print("✅ Balanced Class Distribution:", Counter(y_balanced))


In [ ]:
from tensorflow.keras.layers import Input

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.regularizers import l1
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, mean_squared_error, mean_absolute_error,
    roc_curve, precision_recall_curve, auc
)
from sklearn.utils import resample
from collections import Counter

# -------------------- Split and Augment --------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

def augment(X, y):
    noise = np.random.normal(0, 0.01, X.shape)
    return np.vstack((X, X + noise)), np.hstack((y, y))

X_train, y_train = augment(X_train, y_train)

# -------------------- Undersample Majority Class --------------------
X_flat = X_train.reshape(X_train.shape[0], -1)
X_majority = X_flat[y_train == 0]
X_minority = X_flat[y_train == 1]
y_majority = y_train[y_train == 0]
y_minority = y_train[y_train == 1]

X_majority_down, y_majority_down = resample(
    X_majority, y_majority,
    replace=False,
    n_samples=len(y_minority),
    random_state=42
)

X_balanced = np.vstack((X_majority_down, X_minority))
y_balanced = np.hstack((y_majority_down, y_minority))

# Shuffle and reshape
shuffle_idx = np.random.permutation(len(y_balanced))
X_balanced = X_balanced[shuffle_idx].reshape(-1, X.shape[1], X.shape[2])
y_balanced = y_balanced[shuffle_idx]

print("✅ Balanced Class Distribution:", Counter(y_balanced))

# -------------------- Define Focal Loss --------------------
def focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        return alpha_t * tf.math.pow(1 - p_t, gamma) * bce
    return loss

# -------------------- Define LSTM Models --------------------
def define_models(input_shape):
    return {
        "LSTM_100": Sequential([
            Input(shape=input_shape),
            LSTM(100, return_sequences=True),
            Dropout(0.2),
            LSTM(50),
            Dropout(0.2),
            Dense(25, activation='relu'),
            Dense(1, activation='sigmoid')
        ]),

        "LSTM_50": Sequential([
            Input(shape=input_shape),
            LSTM(50, return_sequences=True),
            Dropout(0.2),
            LSTM(25),
            Dropout(0.2),
            Dense(10, activation='relu'),
            Dense(1, activation='sigmoid')
        ]),

        "LSTM_25_L1": Sequential([
            Input(shape=input_shape),
            LSTM(50, return_sequences=True, kernel_regularizer=l1(0.01)),
            Dropout(0.2),
            LSTM(25, kernel_regularizer=l1(0.01)),
            Dropout(0.2),
            Dense(10, activation='relu', kernel_regularizer=l1(0.01)),
            Dense(1, activation='sigmoid')
        ])
    }
# -------------------- Evaluation Function --------------------
def evaluate_model(y_true, y_pred, model_name="Model"):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auc_score = roc_auc_score(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    print(f"\n📊 {model_name} Performance:")
    print(f"Accuracy : {acc:.4f} | Precision : {prec:.4f}")
    print(f"Recall   : {rec:.4f} | F1-Score  : {f1:.4f}")
    print(f"AUC      : {auc_score:.4f} | RMSE     : {rmse:.4f} | MAE : {mae:.4f}")

    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.show()

    return {
        "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1,
        "AUC": auc_score, "RMSE": rmse, "MAE": mae
    }

# -------------------- Train + Evaluate --------------------
results = {}
models = define_models((X.shape[1], X.shape[2]))

for name, model in models.items():
    print(f"\n🚀 Training {name} ")
    model.compile(optimizer='adam', loss=focal_loss(), metrics=['accuracy'])
    model.fit(X_balanced, y_balanced, epochs=15, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Predictions
    y_pred_prob = model.predict(X_test).flatten()
    fpr, tpr, roc_thresholds = roc_curve(y_test, y_pred_prob)
    precision, recall, pr_thresholds = precision_recall_curve(y_test, y_pred_prob)

    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_threshold = pr_thresholds[np.argmax(f1_scores)]

    # Evaluation at best threshold
    for threshold in [0.5, 0.4, 0.3, 0.2, best_threshold]:
        y_pred = (y_pred_prob >= threshold).astype(int)
        key = f"{name}_thr_{threshold:.2f}"
        results[key] = evaluate_model(y_test, y_pred, key)

    # ROC & PR plots
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(fpr, tpr, label=f"AUC = {auc(fpr, tpr):.4f}")
    plt.title(f"ROC Curve - {name}")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(recall, precision)
    plt.title(f"Precision-Recall Curve - {name}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.tight_layout()
    plt.show()

# -------------------- Summary Table --------------------
results_df = pd.DataFrame(results).T
print("\n✅ Final Evaluation Results:")
print(results_df.round(4))


In [ ]:
!pip install scikit-learn==1.5.0 imbalanced-learn==0.12.0 --force-reinstall --no-deps


In [ ]:
!pip install -U imbalanced-learn

In [ ]:
!pip install -U --force-reinstall --no-deps scikit-learn==1.5.0 imbalanced-learn==0.12.0

In [ ]:
!pip install --force-reinstall --no-deps scikit-learn==1.3.2 imbalanced-learn==0.11.0


In [ ]:
from imblearn.combine import SMOTEENN, SMOTETomek


In [ ]:
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.utils import resample
from collections import Counter
from tensorflow.keras.layers import Input

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, mean_squared_error, mean_absolute_error,
    roc_curve, precision_recall_curve, auc
)

# -------------------- Split and Augment --------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

def augment(X, y):
    noise = np.random.normal(0, 0.01, X.shape)
    return np.vstack((X, X + noise)), np.hstack((y, y))

X_train, y_train = augment(X_train, y_train)

# -------------------- Reshape for Resampling --------------------
X_flat = X_train.reshape(X_train.shape[0], -1)

# -------------------- SMOTEENN --------------------
smote_enn = SMOTEENN(random_state=42)
X_smoteenn, y_smoteenn = smote_enn.fit_resample(X_flat, y_train)
X_smoteenn = X_smoteenn.reshape(-1, X.shape[1], X.shape[2])
print("✅ After SMOTEENN:", Counter(y_smoteenn))

# -------------------- SMOTETomek --------------------
smote_tomek = SMOTETomek(random_state=42)
X_smotetomek, y_smotetomek = smote_tomek.fit_resample(X_flat, y_train)
X_smotetomek = X_smotetomek.reshape(-1, X.shape[1], X.shape[2])
print("✅ After SMOTETomek:", Counter(y_smotetomek))

# -------------------- Manual Undersampling --------------------
X_majority = X_flat[y_train == 0]
X_minority = X_flat[y_train == 1]
y_majority = y_train[y_train == 0]
y_minority = y_train[y_train == 1]

X_majority_down, y_majority_down = resample(
    X_majority, y_majority,
    replace=False,
    n_samples=len(y_minority),
    random_state=42
)

X_manual = np.vstack((X_majority_down, X_minority))
y_manual = np.hstack((y_majority_down, y_minority))

shuffle_idx = np.random.permutation(len(y_manual))
X_manual = X_manual[shuffle_idx].reshape(-1, X.shape[1], X.shape[2])
y_manual = y_manual[shuffle_idx]
print("✅ After Manual Undersampling:", Counter(y_manual))




# -------------------- Define Focal Loss --------------------
def focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        return alpha_t * tf.math.pow(1 - p_t, gamma) * bce
    return loss

# -------------------- Define LSTM Models --------------------
def define_models(input_shape):
    return {
        "LSTM_100": Sequential([
            Input(shape=input_shape),
            LSTM(100, return_sequences=True),
            Dropout(0.2),
            LSTM(50),
            Dropout(0.2),
            Dense(25, activation='relu'),
            Dense(1, activation='sigmoid')
        ]),

        "LSTM_50": Sequential([
            Input(shape=input_shape),
            LSTM(50, return_sequences=True),
            Dropout(0.2),
            LSTM(25),
            Dropout(0.2),
            Dense(10, activation='relu'),
            Dense(1, activation='sigmoid')
        ]),

        "LSTM_25_L1": Sequential([
            Input(shape=input_shape),
            LSTM(50, return_sequences=True, kernel_regularizer=l1(0.01)),
            Dropout(0.2),
            LSTM(25, kernel_regularizer=l1(0.01)),
            Dropout(0.2),
            Dense(10, activation='relu', kernel_regularizer=l1(0.01)),
            Dense(1, activation='sigmoid')
        ])
    }
# -------------------- Evaluation Function --------------------
def evaluate_model(y_true, y_pred, model_name="Model"):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auc_score = roc_auc_score(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    print(f"\n📊 {model_name} Performance:")
    print(f"Accuracy : {acc:.4f} | Precision : {prec:.4f}")
    print(f"Recall   : {rec:.4f} | F1-Score  : {f1:.4f}")
    print(f"AUC      : {auc_score:.4f} | RMSE     : {rmse:.4f} | MAE : {mae:.4f}")

    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.show()

    return {
        "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1,
        "AUC": auc_score, "RMSE": rmse, "MAE": mae
    }


# -------------------- Training and Evaluation --------------------
results = {}
models = define_models((X.shape[1], X.shape[2]))

resampling_methods = {
    "SMOTEENN": (X_smoteenn, y_smoteenn),
    "SMOTETomek": (X_smotetomek, y_smotetomek),
    "ManualUndersample": (X_manual, y_manual)
}


for method, (X_resampled, y_resampled) in resampling_methods.items():
    print(f"\n🔁 Resampling Strategy: {method}")
    for name, model in define_models((X.shape[1], X.shape[2])).items():
        print(f"\n🚀 Training {name} with {method}")
        model.compile(optimizer='adam', loss=focal_loss(), metrics=['accuracy'])
        model.fit(X_resampled, y_resampled, epochs=15, batch_size=32, validation_data=(X_test, y_test), verbose=1)


        

        y_pred_prob = model.predict(X_test).flatten()
        fpr, tpr, roc_thresholds = roc_curve(y_test, y_pred_prob)
        precision, recall, pr_thresholds = precision_recall_curve(y_test, y_pred_prob)

        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_threshold = pr_thresholds[np.argmax(f1_scores)]

        for threshold in [0.5, 0.4, 0.3, 0.2, best_threshold]:
            y_pred = (y_pred_prob >= threshold).astype(int)
            key = f"{method}_{name}_thr_{threshold:.2f}"
            results[key] = evaluate_model(y_test, y_pred, key)

        # ROC & PR plots
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(fpr, tpr, label=f"AUC = {auc(fpr, tpr):.4f}")
        plt.title(f"ROC Curve - {name} ({method})")
        plt.xlabel("FPR")
        plt.ylabel("TPR")
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(recall, precision)
        plt.title(f"Precision-Recall Curve - {name} ({method})")
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.tight_layout()
        plt.show()

# -------------------- Summary Table --------------------
results_df = pd.DataFrame(results).T
print("\n✅ Final Evaluation Results:")
print(results_df.round(4))
